In [8]:
import numpy as np

import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image



 ### init path

In [9]:
# Paramètres de base
base_dir = 'data'
train_dir = os.path.join(base_dir, 'raw_data/train')
test_dir = os.path.join(base_dir, 'raw_data/test')
processed_dir = os.path.join(base_dir, 'processed')
augmented_dir = os.path.join(base_dir, 'processed_data_aug')
target_size = (224, 224)



 ### Create resize file and save as PNY

In [10]:
# Fonction pour redimensionner et sauvegarder les images dans le dossier processed
def resize_and_save_images(source_dir, processed_base):
    for category in os.listdir(source_dir):
        category_dir = os.path.join(source_dir, category)
        processed_category_dir = os.path.join(processed_base, os.path.basename(source_dir), category)
        os.makedirs(processed_category_dir, exist_ok=True)

        for file in os.listdir(category_dir):
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Ajoutez ou retirez des formats selon les besoins
                img_path = os.path.join(category_dir, file)
                img = load_img(img_path, target_size=target_size)
                img_array = img_to_array(img)
                file_name, _ = os.path.splitext(file)
                np.save(os.path.join(processed_category_dir, file_name), img_array)
                


In [11]:
resize_and_save_images(train_dir, processed_dir)
resize_and_save_images(test_dir, processed_dir)

# DATA aug

Test dat aug on few files and adapt params

In [5]:
# Paramètres de data augmentation
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    brightness_range=[0.7, 1.3]
)


In [6]:
# Fonction pour convertir un tableau numpy en image et le sauvegarder au format JPG
def save_array_as_jpg(array, file_path):
    image = Image.fromarray(array.astype('uint8'), 'RGB')
    image.save(file_path)

# Fonction pour appliquer la data augmentation sur des fichiers spécifiques et sauvegarder les images augmentées en tant que JPG
def augment_specific_files_as_jpg(file_paths, augmented_base, num_augmentations=20):
    for file_path in file_paths:
        img_array = np.load(file_path)
        img_array = img_array.reshape((1,) + img_array.shape)

        # Dossier pour sauvegarder les images augmentées
        augmented_dir = os.path.join(augmented_base, os.path.basename(os.path.dirname(file_path)))
        os.makedirs(augmented_dir, exist_ok=True)

        # Créer et sauvegarder des images augmentées
        for i in range(num_augmentations):
            it = datagen.flow(img_array, batch_size=1)
            batch = next(it)
            augmented_image = batch[0]
            jpg_file_path = os.path.join(augmented_dir, f"aug_{i}_{os.path.basename(file_path).replace('.npy', '.jpg')}")
            save_array_as_jpg(augmented_image, jpg_file_path)

In [7]:
# Liste des chemins des fichiers spécifiques
specific_files = [
    'data/processed/test/dog/dog_IMG_0dde90cc4-4d5b-11ea-b58b-0242ac1c0002.npy',
    'data/processed/test/ram/ram_IMG_02d73579a-4d61-11ea-b58b-0242ac1c0002.npy',
    'data/processed/test/hare/hare_IMG_3bb24257e-4d5c-11ea-b58b-0242ac1c0002.npy'
]

# Dossier où sauvegarder les images augmentées
augmented_dir = os.path.join(base_dir, 'analyse_data_aug')

# Appliquer la data augmentation sur les fichiers spécifiques et sauvegarder en tant que JPG
augment_specific_files_as_jpg(specific_files, augmented_dir)
